#  1. Загрузка данных проекта в среду разработки

In [3]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
# Загружаем датасет, знакомимся.
import numpy as np
import pandas as pd
train = pd.read_csv('train.csv', index_col='id')
test = pd.read_csv('test.csv', index_col='id')
print(train)

       Marital status  Application mode  Application order  Course  \
id                                                                   
0                   1                 1                  1    9238   
1                   1                17                  1    9238   
2                   1                17                  2    9254   
3                   1                 1                  3    9500   
4                   1                 1                  2    9500   
...               ...               ...                ...     ...   
76513               1                17                  1    9254   
76514               1                 1                  6    9254   
76515               5                17                  1    9085   
76516               1                 1                  3    9070   
76517               1                 1                  1    9773   

       Daytime/evening attendance  Previous qualification  \
id                          

# 2. Преобразование целевой переменной

In [5]:
# Преобразование целевой переменной. Выясняем, что она многоклассовая
le = LabelEncoder()
train['Target'] = le.fit_transform(train['Target'])
print("Преобразованные значения 'Target':", train['Target'].unique())

Преобразованные значения 'Target': [2 0 1]


# 3. Определение признаков. Повторяем определение признаков из предыдущего задания

In [7]:
# Делим ровно так же, как в задании 2.
categorical_columns = [
    "Marital status", "Application mode", "Application order", "Course",
    "Daytime/evening attendance", "Previous qualification", "Nacionality",
    "Mother's qualification", "Father's qualification", "Mother's occupation",
    "Father's occupation", "Displaced", "Educational special needs", "Debtor",
    "Tuition fees up to date", "Gender", "Scholarship holder", "International"
]
numerical_columns = [
    "Previous qualification (grade)", "Admission grade", "Age at enrollment",
    "Curricular units 1st sem (credited)", "Curricular units 1st sem (enrolled)",
    "Curricular units 1st sem (evaluations)", "Curricular units 1st sem (approved)",
    "Curricular units 1st sem (grade)", "Curricular units 1st sem (without evaluations)",
    "Curricular units 2nd sem (credited)", "Curricular units 2nd sem (enrolled)",
    "Curricular units 2nd sem (evaluations)", "Curricular units 2nd sem (approved)",
    "Curricular units 2nd sem (grade)", "Curricular units 2nd sem (without evaluations)",
    "Unemployment rate", "Inflation rate", "GDP"
]

# 4. Нормализация числовых признаков

In [9]:
# Нормализация числовых признаков
scaler = MinMaxScaler(feature_range=(0, 1))
train[numerical_columns] = scaler.fit_transform(train[numerical_columns])

# 5. Разделение данных на признаки и целевую переменную

In [11]:
# Разделение данных на признаки и целевую переменную
X = train.drop(columns=['Target'])
y = train['Target']

# 6. Диапазон значений 
## Это набор гиперпараметров предназначен для модели XGBoost с задачей многоклассовой классификации
Была выполнена задача по оптимизации поиска параметров из заданных диапазонов с помощью бибилиотеки Hyperopt.

## Основные настройки:
### objective: 'multi:softmax'
Тип задачи — многоклассовая классификация. Модель предсказывает метку класса.

### num_class: 3
Количество классов в задаче.

### random_state: 42
Фиксирует случайность для воспроизводимости результатов.

## Настройки структуры деревьев:
### max_depth:
Максимальная глубина каждого дерева. Более глубокие деревья могут моделировать сложные зависимости, но риск переобучения увеличивается.

### min_child_weight:
Минимальный вес суммарного градиента в дочерних узлах. Увеличение значения делает модель более консервативной.

### max_delta_step:
Максимальное изменение весов деревьев за одну итерацию. Помогает стабилизировать обучение, особенно при дисбалансе классов.

## Настройки скорости обучения
### learning_rate:
Скорость обучения. Баланс между количеством деревьев и устойчивостью модели. Меньшие значения требуют большего числа итераций.

### n_estimators:
Количество деревьев в ансамбле

## Настройки выборки данных и признаков:
### colsample_bytree:
Доля признаков, используемых для построения каждого дерева.

### colsample_bylevel:
Доля признаков, выбираемых на каждом уровне дерева.

### colsample_bynode:
Доля признаков для выбора на уровне каждого узла.

### subsample:
Доля обучающих данных, используемых для каждого дерева. Это снижает переобучение.


In [22]:
from sklearn.preprocessing import LabelEncoder

best_params = {
    'objective': 'multi:softmax',
    'num_class': 3,  # Количество классов
    'random_state': 42,
    'max_depth': 12, 
    'learning_rate': 0.1380959235295076, 
    'n_estimators': 229, 
    'colsample_bytree': 0.9428442050675058, 
    'min_child_weight': 4, 
    'subsample': 0.9200515940656013, 
    'colsample_bylevel': 0.6455519399349366, 
    'colsample_bynode': 0.6808130293292364, 
    'max_delta_step': 27
}


# 7. Применяем SMOTE для устранения дисбаланса классов до разделения данных на обучающую и тестовою выборки, чтобы избежать утечки.

In [15]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

#Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42
)

# 8. Cоздаем и обучаем модель с найденными параметрами test)

In [17]:
#  Обучение модели
final_model = XGBClassifier(**best_params)
final_model.fit(X_train, y_train)

#  Прогнозируем на тестовой выборке
y_pred = final_model.predict(X_test)

# 9. Оцениваем точность модель

In [18]:
# Оцениваем точность
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Вывод отчета по классификации
print("Classification Report:")
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Вывод матрицы путаницы
conf_matrix = confusion_matrix(y_test, y_pred)

# Для удобства можно добавить строки с метками классов
print("Confusion Matrix with Class Labels:")
for i, row in enumerate(conf_matrix):
    print(f"Class {i}: {row}")
    
    
# Рассчитываем метрики
classes = range(best_params['num_class'])
print("\nClass-wise Metrics:")
for cls in classes:
    TP = conf_matrix[cls, cls]
    FP = conf_matrix[:, cls].sum() - TP
    FN = conf_matrix[cls, :].sum() - TP
    TN = conf_matrix.sum() - (TP + FP + FN)

    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0

    print(f"Class {cls}:")
    print(f"  Sensitivity (Recall): {sensitivity:.4f}")
    print(f"  Specificity: {specificity:.4f}")
    print(f"  Precision: {precision:.4f}")

Accuracy: 0.870509875976114
Classification Report:
Confusion Matrix with Class Labels:
Class 0: [6200  690  333]
Class 1: [ 437 5993  771]
Class 2: [ 120  468 6758]

Class-wise Metrics:
Class 0:
  Sensitivity (Recall): 0.8584
  Specificity: 0.9617
  Precision: 0.9176
Class 1:
  Sensitivity (Recall): 0.8322
  Specificity: 0.9205
  Precision: 0.8381
Class 2:
  Sensitivity (Recall): 0.9200
  Specificity: 0.9235
  Precision: 0.8596


# В целом, удалось добиться увеличения точности модели с 0.827 (без гиперпараметров и балансировкой, результат был выявлен на первых тестах, они не вошли в этот Notebook) до ≈ 0.8705 (с гиперпараметрами и балансировкой классов). Подобная модель была так же применена и на данных по итогам Спринта 2 (отбор топ-20 признаков, введение новых признаков), но она показала немного худший результат- 0.8542.